In [1]:
import streamlit as st
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, label_binarize

In [2]:
# Load the datasets
X_train_binary = pd.read_csv('Binary/Model/Binary_Train_Data.csv')
X_test_binary = pd.read_csv('Binary/Model/Binary_Test_Data.csv')
X_train_multi = pd.read_csv('Multi/Model/Multi_Train_Data.csv')
X_test_multi = pd.read_csv('Multi/Model/Multi_Test_Data.csv')

In [3]:
# Standardize the datasets
binary_scaler = StandardScaler()
multi_scaler = StandardScaler()
X_train_binary_scaled = binary_scaler.fit_transform(X_train_binary)
X_test_binary_scaled = binary_scaler.transform(X_test_binary)
X_train_multi_scaled = multi_scaler.fit_transform(X_train_multi)
X_test_multi_scaled = multi_scaler.transform(X_test_multi)

In [4]:
y_test_binary = pd.read_csv('Binary/Model/y_test_binary.csv')['at_risk_binary_encoded'].values
y_test_multi = pd.read_csv('Multi/Model/y_test_multi.csv')['at_risk_multi_encoded'].values

In [5]:
# Load the binary models
binary_model_files = [
    'Binary/Model/Base/lr_model_binary.pkl',
    'Binary/Model/Tuned/best_dt_model_binary.pkl',
    'Binary/Model/Tuned/best_rf_model_binary.pkl',
    'Binary/Model/Base/gb_model_binary.pkl',
    'Binary/Model/Tuned/best_xgb_model_binary.pkl'
]

binary_model_names = ['Base Logistic Regression', 'Tuned Decision Tree', 'Tuned Random Forest', 'Base Gradient Boosting', 'Tuned XGBoost']
binary_models = [joblib.load(file_path) for file_path in binary_model_files]
ann_model_binary = tf.keras.models.load_model('Binary/Model/Tuned/best_ann_model_binary.keras')


In [6]:
# Load the multi-class models
multi_model_files = [
    'Multi/Model/Tuned/best_lr_model_multi.pkl',
    'Multi/Model/Tuned/best_dt_model_multi.pkl',
    'Multi/Model/Tuned/best_rf_model_multi.pkl',
    'Multi/Model/Tuned/best_gb_model_multi.pkl',
    'Multi/Model/Tuned/best_xgb_model_multi.pkl'
]

multi_model_names = ['Tuned Logistic Regression', 'Tuned Decision Tree', 'Tuned Random Forest', 'Tuned Gradient Boosting', 'Tuned XGBoost']
multi_models = [joblib.load(file_path) for file_path in multi_model_files]
ann_model_multi = tf.keras.models.load_model('Multi/Model/Tuned/best_ann_model_multi.keras')

In [7]:
# Define evaluation functions
def evaluate_model(model, X_test, y_test, is_binary=True):
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)
    
    if is_binary:
        y_pred_prob = y_pred_prob[:, 1]
    
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision (Class 0)': precision_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else precision_score(y_test, y_pred, labels=[0], average=None)[0],
        'Precision (Class 1)': precision_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else precision_score(y_test, y_pred, labels=[1], average=None)[0],
        'Precision (Class 2)': '-' if is_binary else (precision_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'Recall (Class 0)': recall_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else recall_score(y_test, y_pred, labels=[0], average=None)[0],
        'Recall (Class 1)': recall_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else recall_score(y_test, y_pred, labels=[1], average=None)[0],
        'Recall (Class 2)': '-' if is_binary else (recall_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'F1 Score (Class 0)': f1_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else f1_score(y_test, y_pred, labels=[0], average=None)[0],
        'F1 Score (Class 1)': f1_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else f1_score(y_test, y_pred, labels=[1], average=None)[0],
        'F1 Score (Class 2)': '-' if is_binary else (f1_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'ROC AUC (Class 0)': roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 0], y_pred_prob[:, 0]) if not is_binary else roc_auc_score(y_test, y_pred_prob),
        'ROC AUC (Class 1)': roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 1], y_pred_prob[:, 1]) if not is_binary else '-',
        'ROC AUC (Class 2)': '-' if is_binary else (roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 2], y_pred_prob[:, 2]) if len(np.unique(y_test)) > 2 else '-')
    }
    return metrics, y_pred_prob, y_pred

In [8]:
def evaluate_ann_model(model, X_test, y_test, is_binary=True):
    y_pred_prob = model.predict(X_test)
    
    if is_binary:
        # Ensure the probabilities are in the correct shape
        if y_pred_prob.shape[1] == 1:
            y_pred_prob = np.hstack([1 - y_pred_prob, y_pred_prob])  # Add the second column for binary classification
        y_pred = (y_pred_prob[:, 1] > 0.5).astype(int)
        y_pred_prob = y_pred_prob[:, 1]
    else:
        y_pred = y_pred_prob.argmax(axis=1)
    
    metrics = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision (Class 0)': precision_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else precision_score(y_test, y_pred, labels=[0], average=None)[0],
        'Precision (Class 1)': precision_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else precision_score(y_test, y_pred, labels=[1], average=None)[0],
        'Precision (Class 2)': '-' if is_binary else (precision_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'Recall (Class 0)': recall_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else recall_score(y_test, y_pred, labels=[0], average=None)[0],
        'Recall (Class 1)': recall_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else recall_score(y_test, y_pred, labels=[1], average=None)[0],
        'Recall (Class 2)': '-' if is_binary else (recall_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'F1 Score (Class 0)': f1_score(y_test, y_pred, pos_label=0, average='binary') if is_binary else f1_score(y_test, y_pred, labels=[0], average=None)[0],
        'F1 Score (Class 1)': f1_score(y_test, y_pred, pos_label=1, average='binary') if is_binary else f1_score(y_test, y_pred, labels=[1], average=None)[0],
        'F1 Score (Class 2)': '-' if is_binary else (f1_score(y_test, y_pred, labels=[2], average=None)[0] if len(np.unique(y_test)) > 2 else '-'),
        'ROC AUC (Class 0)': roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 0], y_pred_prob[:, 0]) if not is_binary else roc_auc_score(y_test, y_pred_prob),
        'ROC AUC (Class 1)': roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 1], y_pred_prob[:, 1]) if not is_binary else '-',
        'ROC AUC (Class 2)': '-' if is_binary else (roc_auc_score(label_binarize(y_test, classes=[0, 1, 2])[:, 2], y_pred_prob[:, 2]) if len(np.unique(y_test)) > 2 else '-')
    }
    return metrics, y_pred_prob, y_pred

In [9]:
# Evaluate binary models
binary_results = []
binary_pred_probs = []
binary_preds = []

for model, name in zip(binary_models, binary_model_names):
    metrics, y_pred_prob, y_pred = evaluate_model(model, X_test_binary_scaled, y_test_binary, is_binary=True)
    metrics['Model'] = name
    metrics['Type'] = 'Binary'
    binary_results.append(metrics)
    binary_pred_probs.append((name, y_pred_prob))
    binary_preds.append((name, y_pred))

ann_metrics, y_pred_prob_ann, y_pred_ann = evaluate_ann_model(ann_model_binary, X_test_binary_scaled, y_test_binary, is_binary=True)
ann_metrics['Model'] = 'Tuned ANN'
ann_metrics['Type'] = 'Binary'
binary_results.append(ann_metrics)
binary_pred_probs.append(('Tuned ANN', y_pred_prob_ann))
binary_preds.append(('Tuned ANN', y_pred_ann))

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step


In [10]:
# Evaluate multi-class models
multi_results = []
multi_pred_probs = []
multi_preds = []

for model, name in zip(multi_models, multi_model_names):
    metrics, y_pred_prob, y_pred = evaluate_model(model, X_test_multi_scaled, y_test_multi, is_binary=False)
    metrics['Model'] = name
    metrics['Type'] = 'Multi-Class'
    multi_results.append(metrics)
    multi_pred_probs.append((name, y_pred_prob))
    multi_preds.append((name, y_pred))

ann_metrics, y_pred_prob_ann, y_pred_ann = evaluate_ann_model(ann_model_multi, X_test_multi_scaled, y_test_multi, is_binary=False)
ann_metrics['Model'] = 'Tuned ANN'
ann_metrics['Type'] = 'Multi-Class'
multi_results.append(ann_metrics)
multi_pred_probs.append(('Tuned ANN', y_pred_prob_ann))
multi_preds.append(('Tuned ANN', y_pred_ann))

204/204 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [11]:
# Combine results
combined_results = pd.DataFrame(binary_results + multi_results)

In [12]:
# Reorder the metrics
desired_order = [
    'Accuracy',
    'Precision (Class 0)', 'Precision (Class 1)', 'Precision (Class 2)',
    'Recall (Class 0)', 'Recall (Class 1)', 'Recall (Class 2)',
    'F1 Score (Class 0)', 'F1 Score (Class 1)', 'F1 Score (Class 2)',
    'ROC AUC (Class 0)', 'ROC AUC (Class 1)', 'ROC AUC (Class 2)'
]

# Define the desired order of models within each type
binary_model_order = ['Base Logistic Regression', 'Tuned Decision Tree', 'Tuned Random Forest', 'Base Gradient Boosting', 'Tuned XGBoost', 'Tuned ANN']
multi_model_order = ['Tuned Logistic Regression', 'Tuned Decision Tree', 'Tuned Random Forest', 'Tuned Gradient Boosting', 'Tuned XGBoost', 'Tuned ANN']

# Transform the DataFrame to have metrics as rows and models as columns
combined_results_pivot = combined_results.melt(id_vars=['Type', 'Model'], var_name='Metric', value_name='Value')
combined_results_pivot = combined_results_pivot.pivot_table(index='Metric', columns=['Type', 'Model'], values='Value', aggfunc='first')

# Reindex the DataFrame to the desired order
combined_results_pivot = combined_results_pivot.reindex(desired_order)

# Reorder the columns within each type based on the desired model order
binary_columns = [col for col in binary_model_order if col in combined_results_pivot['Binary'].columns]
multi_columns = [col for col in multi_model_order if col in combined_results_pivot['Multi-Class'].columns]

# Combine the ordered columns
ordered_columns = [('Binary', col) for col in binary_columns] + [('Multi-Class', col) for col in multi_columns]

# Reindex the DataFrame columns
combined_results_pivot = combined_results_pivot.reindex(columns=pd.MultiIndex.from_tuples(ordered_columns))

# Center-align the text in the DataFrame and add border between types
styled_results = combined_results_pivot.style.set_properties(**{'text-align': 'center'})
styled_results = styled_results.set_table_styles([
    dict(selector='th', props=[('text-align', 'center'), ('font-weight', 'bold')]),
    dict(selector='td', props=[('border', '1px solid black'), ('text-align', 'center')]),
    dict(selector='th', props=[('border', '1px solid black'), ('text-align', 'center')])
])

# Apply thick border between types
styled_results = styled_results.set_table_styles([
    {'selector': 'td:nth-child(6)', 'props': [('border-right', '3px solid black')]},  # Adjust based on the column position
    {'selector': 'th:nth-child(6)', 'props': [('border-right', '3px solid black')]}   # Adjust based on the column position
], overwrite=False)

# Bold the column names including the 'Type'
styled_results = styled_results.set_table_styles([
    {'selector': 'th.col_heading', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'th.row_heading', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
    {'selector': 'th.level0', 'props': [('font-weight', 'bold'), ('text-align', 'center')]}  # Bold the 'Type' level header
], overwrite=False)

# Apply full border
styled_results = styled_results.set_table_styles([
    dict(selector='th', props=[('border', '1px solid black'), ('text-align', 'center')]),
    dict(selector='td', props=[('border', '1px solid black'), ('text-align', 'center')])
])

# Display the results
print("Model Performance Metrics:")
display(styled_results)

import dataframe_image as dfi
# Save the styled DataFrame as an image
dfi.export(styled_results, 'model_performance_metrics.png')

print("Model Performance Metrics have been saved as an image.")

Model Performance Metrics:


Model Performance Metrics have been saved as an image.
